In [1]:
%run "libraries.ipynb"

In [2]:
hyperlinks_graph = nx.read_gexf("data/hyperlinks.gexf")

In [3]:
g = nx.DiGraph()

nb_out = 3

for u in hyperlinks_graph:
    nb = sorted(hyperlinks_graph[u].items(), key=lambda x: -x[1]["term occurence"])
    
    # filtering self loops
    nb = [ n for n in nb if n[0] != u ]

    for v in nb[0:nb_out]:
        #print v
        g.add_edge(u, v[0], attr_dict=v[1])

In [4]:
print "reduced graph"
print "============="
print "nodes: %s" % (len(g.nodes()))
print "edges: %s" % (len(g.edges()))
print "reduction: %s/%s (%s)" % (len(g.edges()),
                                 len(hyperlinks_graph.edges()),
                                 float(len(g.edges()))/float(len(hyperlinks_graph.edges())))

reduced graph
nodes: 299
edges: 865
reduction: 865/3781 (0.228775456229)


In [5]:
nx.write_gexf(g, "data/reading_maps/pages-hyperlinks-reduced-3.gexf")